In [11]:
from IPython.display import display

In [12]:
import pandas as pd
import numpy as np
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

from embedding import get_embedding, cos_sim
import breakdown

In [19]:
def prepare_dataframe(csv):
    df = pd.read_csv(csv)
    df["embedded"] = df["embedded"].map(lambda x: eval(x))
    return df

In [14]:
class Sentence:
    def __init__(self, verb, agent, role, team):
        self.verb = verb
        self.agent = agent
        self.role = role
        self.team = team

In [22]:

def to_vectors(talker, comment):
    # 要素に分割
    sentences = eval(breakdown.get_list(talker, comment)) [0]

    # 各要素をembeddingしたリスト
    return [get_embedding(sentence) for sentence in sentences]

In [24]:
talker = "Agent[00]"
comment = "Agent[00]を占ったら黒だったよ。Agent[01]を占ったら白だったよ。"
print(eval(breakdown.get_list(talker, comment))[0])

['I was black when inspected', 'Agent[01] was white when inspected']


In [27]:
#最大類似度とそのindexをタプルに入れて返す関数

def ruizido(df, vector):
    """
    vector: 入力文の要素の一つをembedしたもの
    """
    similarity_list = [] #一旦、類似度だけを格納しておくためのリスト
    for i in df["embedded"]:
        similarity_list.append(cos_sim(i, vector))
        
    a = similarity_list.index(max(similarity_list))
    b = max(similarity_list)
    
    
    #最大の類似度と、その文のindexを返す
    return (b, a)

In [40]:
#最終的なリストの作成
#from send import Sentence

def infolist(df, vectorlist): #vectorlistはto_vectors(talker, comment)の返り値
    a = []
    for i in vectorlist:
        verb = df.loc[ruizido(df, i)[1], "Verb"]
        agent = df.loc[ruizido(df, i)[1], "Agent"]
        role = df.loc[ruizido(df, i)[1], "Role"]
        team = df.loc[ruizido(df, i)[1], "Team"]
        
        
        #a.append((verb, agent, role, team))
        
        sentence = Sentence(verb, agent, role, team)
        a.append(sentence)
    
    return a
        
    
        

In [41]:
df = prepare_dataframe("new_new_embedded_all.csv")
talker = "Agent[00]"
comment = "Agent[00]を占ったら黒だったよ。Agent[01]を占ったら白だったよ。"

print(infolist(df, to_vectors(talker, comment)))






[<__main__.Sentence object at 0x000001DBEAC1BAA0>, <__main__.Sentence object at 0x000001DBEACA9190>]
